In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

kafka_bootstrap_servers = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
kafka_topic = 'github_logs'
api_key = 'LLPDTJOM6HZMVVEK'
api_secret = 'M35bZlxNWayWwn8p8F/4nxbBrsdt3TOhSAM5BTCK48C4FZjiFeHckPhD2ZCOBWiF'

date_ = datetime.utcnow().strftime("%Y-%m-%d")
bronze_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/githublogs/bronze/{date_}"

In [0]:
df_raw = spark.readStream.format('kafka') \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers)  \
    .option("subscribe", kafka_topic) \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config", f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="{api_key}" password="{api_secret}";') \
    .option("startingOffsets", 'earliest') \
    .load()

df_raw = df_raw.selectExpr("CAST(value as STRING) as json_str") 

In [0]:
df_schema = StructType() \
    .add("timestamp", StringType()) \
    .add("event_type", StringType()) \
    .add("repo", StringType()) \
    .add("actor", StringType()) \
    .add("raw", StringType())

df_bronze = df_raw.select(
    from_json(col("json_str"), df_schema).alias("data")
    ).select("data.*")

In [0]:
checkpoint_path = f"/tmp/checkpoints/github_logs/{date_}"

dbutils.fs.rm("/tmp/checkpoints/githublogs-bronze/", True)

query = df_bronze.writeStream \
    .format("delta") \
    .option("checkpointLocation", f"/tmp/checkpoints/githublogs-bronze/{date_}/")\
    .option("path", bronze_path) \
    .trigger(availableNow=True) \
    .start()
query.awaitTermination()
